In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score

In [3]:
# Load the data
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']
dataframes = []
for year in years:
    data = pd.read_csv(f"EngineeringRanking_{year}.csv")
    dataframes.append(data)
data = pd.concat(dataframes)

FileNotFoundError: [Errno 2] No such file or directory: 'EngineeringRanking_2016.csv'

In [4]:
# Combine data with EngineeringRanking.csv
combined_data = pd.read_csv("EngineeringRanking.csv")
data = pd.concat([data, combined_data])

FileNotFoundError: [Errno 2] No such file or directory: 'EngineeringRanking.csv'

In [ ]:
# Handling missing values by filling with 0
data.fillna(0, inplace=True)

In [ ]:
# Convert non-numeric columns to numeric types
numeric_columns = ['Score', 'Rank', 'TLR', 'RPC', 'GO', 'OI', 'Perception']
data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Remove rows with missing target variable (Rank_21)
data.dropna(subset=['Rank_21'], inplace=True)

In [ ]:
# Feature selection
features = ['Score_21', 'TLR_21', 'RPC_21', 'GO_21', 'OI_21', 'Perception_21']
X = data[features]
y = data['Rank_21']  # Target variable for prediction

In [ ]:
# Splitting data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
### Feature Scaling
'''
## Min-Max Scaling:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

'''
## Standardization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
###Checking Test Data for Missing Values

# Concatenate X_test and y_test into a single DataFrame
test_data = pd.concat([X_test, y_test], axis=1)

# Checking for missing values in the entire test data
missing_values_test = test_data.isnull().sum()
print("Missing values in test data:")
print(missing_values_test)

Missing values in test data:
Score_21         0
TLR_21           0
RPC_21           0
GO_21            0
OI_21            0
Perception_21    0
Rank_21          0
dtype: int64


In [ ]:
# Train the model on scaled data
model = XGBRegressor()
model.fit(X_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
# Validation on 2023 data
combined_data.dropna(subset=['Rank_21'], inplace=True)  # Remove rows with missing target values
X_val = combined_data[features]
y_val_true = combined_data['Rank_21']

# Feature scaling on validation data
X_val_scaled = scaler.transform(X_val)

In [ ]:
# Evaluate the model on the training set
y_train_pred = model.predict(X_train_scaled)
train_mse = mean_squared_error(y_train, y_train_pred)
print(f"Mean Squared Error on Training Set: {train_mse}")

Mean Squared Error on Training Set: 3.4356926394838174e-07


In [ ]:
# Evaluate the model on the validation set
y_val_pred = model.predict(X_val_scaled)
val_mse = mean_squared_error(y_val_true, y_val_pred)
print(f"Mean Squared Error on Validation Set (2023 data): {val_mse}")

Mean Squared Error on Validation Set (2023 data): 0.9645161011927166


In [ ]:
# Evaluate the model on the test set
y_test_pred = model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, y_test_pred)
print(f"Mean Squared Error on Test Set: {test_mse}")

Mean Squared Error on Test Set: 0.7014648861665463


In [ ]:
# Check for overfitting or underfitting
if train_mse < val_mse:
    print("The model might be overfitting.")
elif train_mse > val_mse:
    print("The model might be underfitting.")
else:
    print("The model is likely properly fitted.")

The model might be overfitting.


In [ ]:
# Get predictions for 2023 data
combined_data['Predicted_Rank_2023'] = y_val_pred
sorted_data_2023 = combined_data.sort_values(by='Predicted_Rank_2023')

In [ ]:
#Accuracy:

# Calculate accuracy
threshold = 1  # Define the threshold for accurate predictions (e.g., within ±1 rank)

# Count the number of accurate predictions
accurate_predictions = np.sum(np.abs(y_val_true - y_val_pred) <= threshold)

# Total number of predictions
total_predictions = len(y_val_true)

# Calculate accuracy
accuracy = (accurate_predictions / total_predictions) * 100
print(f"Accuracy on Validation Set (2023 data): {accuracy:.2f}%")


Accuracy on Validation Set (2023 data): 87.50%


In [ ]:
# DataFrame to compare actual and predicted ranks
comparison_df = pd.DataFrame({
    'Institute Name': combined_data['Institute Name'],
    'City': combined_data['City'],
    'State': combined_data['State'],
    'Actual Rank 2023': y_val_true,
    'Predicted Rank 2023': y_val_pred.round().astype(int)  # Round predicted ranks to the nearest integer
})

# Display the comparison DataFrame
print("Comparison of Actual and Predicted Ranks for 2023:")
print(comparison_df)


Comparison of Actual and Predicted Ranks for 2023:
                                 Institute Name       City           State  \
0         Indian Institute of Technology Madras    Chennai      Tamil Nadu   
1          Indian Institute of Technology Delhi  New Delhi           Delhi   
2         Indian Institute of Technology Bombay     Mumbai     Maharashtra   
3         Indian Institute of Technology Kanpur     Kanpur   Uttar Pradesh   
4      Indian Institute of Technology Kharagpur  Kharagpur     West Bengal   
..                                          ...        ...             ...   
195       The National Institute of Engineering     Mysore       Karnataka   
196        K. J. Somaiya College of Engineering     Mumbai     Maharashtra   
197  Kakatiya Institute of Technology & Science   Warangal       Telangana   
198             Walchand College of Engineering     Sangli     Maharashtra   
199                 Sri Venkateswara University   Tirupati  Andhra Pradesh   

     Actual 

**CLUSTERING ALGORITHM**

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/Datasets/EngineeringRanking.csv")
data.fillna(0, inplace=True)
data.tail()

,Institute Id,Institute Name,City,State,Score_21,Rank_21,TLR_21,RPC_21,GO_21,OI_21,...,GO_17,OI_17,Perception_17,Score_16,Rank_16,TLR_16,RPC_16,GO_16,OI_16,Perception_16
268,IR17-ENGG-1-26228,C.V. Raman College of Engineering,Bhubneshwar,Odisha,0.0,0.0,0.0,0.0,0.0,0.0,...,49.95,72.48,21.66,0.0,0,0.0,0.0,0.0,0.0,0.0
269,IR17-ENGG-2-10476,"Maharashtra Institute of Technology, Pune",Pune,Maharashtra,0.0,0.0,0.0,0.0,0.0,0.0,...,52.13,63.39,1.46,0.0,0,0.0,0.0,0.0,0.0,0.0
270,IR17-ENGG-2-12411,Sri Sai Ram Engineering College,Chennai,Tamil Nadu,0.0,0.0,0.0,0.0,0.0,0.0,...,59.32,68.75,4.86,0.0,0,0.0,0.0,0.0,0.0,0.0
271,IR17-ENGG-2-12581,ST. Joseph's College of Engineering,Chennai,Tamil Nadu,0.0,0.0,0.0,0.0,0.0,0.0,...,49.39,64.92,2.80,0.0,0,0.0,0.0,0.0,0.0,0.0
272,IR17-ENGG-2-1-2810997882,K.S.Rangasamy College of Technology,Tiruchengode,Tamil Nadu,0.0,0.0,0.0,0.0,0.0,0.0,...,46.44,60.98,5.85,0.0,0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def display_city_list(state):
    cities = data[data['State'] == state]['City'].unique()
    print("Cities in", state, "are:")
    for city in cities:
        print(city)

def display_institutes_by_city(city):
    city_data = data[data['City'] == city]
    if len(city_data) == 0:
        print("No institutes found in", city)
        return
    print("\nInstitutes in", city, "from 2016 to 2021:")
    print('-' * 100)
    print("{:<40} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(
        "Institute Name", "2016", "2017", "2018", "2019", "2020", "2021"))
    print('-' * 100)
    for index, row in city_data.iterrows():
        print("{:<40} {:<10} {:<10} {:<10} {:<10} {:<10} {:<10}".format(
            row['Institute Name'],
            row['Rank_16'], row['Rank_17'], row['Rank_18'], row['Rank_19'], row['Rank_20'], row['Rank_21']
        ))
    print('-' * 100)

# Main program
state = input("Enter state: ")
display_city_list(state)
city = input("Enter city from the list: ")
display_institutes_by_city(city)

Enter state: Maharashtra
Cities in Maharashtra are:
Mumbai
Nagpur
Pune
Sangli
Navi Mumbai
Mumbai Suburban
Enter city from the list: Mumbai

Institutes in Mumbai from 2016 to 2021:
----------------------------------------------------------------------------------------------------
Institute Name                           2016       2017       2018       2019       2020       2021      
----------------------------------------------------------------------------------------------------
Indian Institute of Technology Bombay    2          2.0        2          3.0        3.0        3.0       
Institute of Chemical Technology         0          14.0       10         12.0       18.0       15.0      
Veermata Jijabai Technological Institute,  (VJTI, Mumbai) 84         0.0        95         105.0      71.0       82.0      
Bharatiya Vidya Bhavan`s Sardar Patel Institute of Technology 0          0.0        0          114.0      125.0      143.0     
SVKM`s Narsee Monjee Institute of Management 